<a href="https://colab.research.google.com/github/MitjaGo/BAR/blob/main/ID3TAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================================
# 🧩 INSTALL DEPENDENCIES
# ===========================================
!pip install eyed3 ipywidgets pillow

import eyed3
from google.colab import files
from IPython.display import display, Image
import ipywidgets as widgets
import zipfile
from datetime import datetime
import re
from PIL import Image as PILImage
import io

# ===========================================
# 📸 STEP 1: Upload default thumbnail
# ===========================================
print("📸 Upload a default JPG/PNG thumbnail for all MP3s:")
image_upload = files.upload()
img_name, default_img_data = next(iter(image_upload.items()))
display(Image(data=default_img_data, width=150))

# Resize image for compatibility
def resize_jpeg(img_data, max_size=500):
    img = PILImage.open(io.BytesIO(img_data))
    img.thumbnail((max_size, max_size))
    out = io.BytesIO()
    img.convert("RGB").save(out, format='JPEG', quality=85)
    return out.getvalue()

default_img_data = resize_jpeg(default_img_data)

# ===========================================
# 🎵 STEP 2: Upload MP3 files
# ===========================================
print("\n⬆️ Upload up to 50 MP3 files:")
upload = files.upload()
mp3_files = list(upload.keys())[:50]
print(f"✅ Uploaded {len(mp3_files)} MP3 files.")

# ===========================================
# 🖼️ Helper: Embed thumbnail
# ===========================================
def embed_thumbnail_eyed3(mp3_file, image_data):
    audiofile = eyed3.load(mp3_file)
    if audiofile.tag is None:
        audiofile.initTag()
    audiofile.tag.images.set(3, image_data, "image/jpeg", u"Cover")
    audiofile.tag.save(version=eyed3.id3.ID3_V2_3)

# Embed default thumbnail into all files
for f in mp3_files:
    embed_thumbnail_eyed3(f, default_img_data)
print("🖼️ Default thumbnail embedded into all MP3 files.")

# ===========================================
# 💿 STEP 3: Bulk Album Edit
# ===========================================
bulk_album_widget = widgets.Text(
    value='',
    description='Set Album for All:',
    layout=widgets.Layout(width='300px')
)
bulk_album_btn = widgets.Button(description='Apply to All', button_style='warning')
display(widgets.HBox([bulk_album_widget, bulk_album_btn]))

# ===========================================
# 🧠 STEP 4: Parse filename to extract Artist & Title
# ===========================================
def parse_filename(filename):
    """
    Example: '01 - Mina - Oggi Sono Io.mp3'
    → ('Mina', 'Oggi Sono Io')
    """
    name = re.sub(r'\.\w+$', '', filename)  # remove extension
    parts = re.split(r'\s*-\s*', name)
    parts = [p.strip() for p in parts if p.strip()]

    artist = ""
    title = ""

    if len(parts) >= 3:
        # e.g. ['01', 'Mina', 'Oggi Sono Io']
        artist = parts[1]
        title = " - ".join(parts[2:])
    elif len(parts) == 2:
        # e.g. ['Mina', 'Oggi Sono Io']
        artist, title = parts
    else:
        title = parts[0] if parts else filename

    return artist, title

# ===========================================
# 📝 STEP 5: Create editable widgets per file
# ===========================================
file_widgets = []

for f in mp3_files:
    audiofile = eyed3.load(f)
    if audiofile.tag is None:
        audiofile.initTag()

    # Try to parse filename if tags missing or empty
    parsed_artist, parsed_title = parse_filename(f)

    title = audiofile.tag.title or parsed_title or ""
    artist = audiofile.tag.artist or parsed_artist or ""
    album = audiofile.tag.album or ""

    title_widget = widgets.Text(value=title, description='Title:', layout=widgets.Layout(width='250px'))
    artist_widget = widgets.Text(value=artist, description='Artist:', layout=widgets.Layout(width='250px'))
    album_widget = widgets.Text(value=album, description='Album:', layout=widgets.Layout(width='250px'))
    img_widget = widgets.Image(value=default_img_data, format='jpeg', width=80, height=80)

    new_img_data = {'data': None}

    # Upload per-file thumbnail
    def make_upload_callback(img_w, store):
        def on_upload(change):
            uploaded = files.upload()
            if uploaded:
                _, data = next(iter(uploaded.items()))
                data_resized = resize_jpeg(data)
                img_w.value = data_resized
                store['data'] = data_resized
        return on_upload

    upload_btn = widgets.Button(description="📤 Change Thumbnail", layout=widgets.Layout(width='150px'))
    upload_btn.on_click(make_upload_callback(img_widget, new_img_data))

    hbox = widgets.HBox([
        img_widget,
        widgets.Label(f, layout=widgets.Layout(width='200px')),
        title_widget,
        artist_widget,
        album_widget,
        upload_btn
    ])

    file_widgets.append((f, title_widget, artist_widget, album_widget, img_widget, new_img_data, hbox))

display(widgets.Label("📝 Edit MP3 Tags and optionally upload new thumbnails:"))
for w in file_widgets:
    display(w[-1])

# ===========================================
# 💿 Bulk Album Button Function
# ===========================================
def on_bulk_album_clicked(b):
    album_name = bulk_album_widget.value.strip()
    for _, _, _, album_w, _, _, _ in file_widgets:
        album_w.value = album_name

bulk_album_btn.on_click(on_bulk_album_clicked)

# ===========================================
# 💾 STEP 6: Save & Download
# ===========================================
save_btn = widgets.Button(description="💾 Save All Tags", button_style='success')
zip_btn  = widgets.Button(description="⬇️ Download All MP3s as ZIP", button_style='info')
output   = widgets.Output()
display(save_btn, zip_btn, output)

def clean_filename(filename):
    # remove trailing "(1)" or similar duplicates
    return re.sub(r"\s*\(\d+\)(?=\.\w+$)", "", filename)

# Save all button
def on_save_clicked(b):
    for f, title_w, artist_w, album_w, img_w, new_img, _ in file_widgets:
        audiofile = eyed3.load(f)
        if audiofile.tag is None:
            audiofile.initTag()

        audiofile.tag.title = title_w.value.strip()
        audiofile.tag.artist = artist_w.value.strip()
        audiofile.tag.album = album_w.value.strip()

        image_data = new_img['data'] if new_img['data'] else default_img_data
        audiofile.tag.images.set(3, image_data, "image/jpeg", u"Cover")

        audiofile.tag.save(version=eyed3.id3.ID3_V2_3)

    with output:
        output.clear_output()
        print("✅ All MP3 tags and thumbnails updated successfully!")

# ZIP button
def on_zip_clicked(b):
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_name = f"edited_mp3s_{now}.zip"
    with zipfile.ZipFile(zip_name, "w") as z:
        for f, *_ in file_widgets:
            new_name = clean_filename(f)
            z.write(f, arcname=new_name)
    files.download(zip_name)

save_btn.on_click(on_save_clicked)
zip_btn.on_click(on_zip_clicked)


In [ ]:
# ===========================================
# 🧩 INSTALL DEPENDENCIES
# ===========================================
!pip install eyed3 ipywidgets pillow

import eyed3
from google.colab import files
from IPython.display import display, Image
import ipywidgets as widgets
import zipfile
from datetime import datetime
import re
from PIL import Image as PILImage, UnidentifiedImageError
import io
import os

# ===========================================
# ✅ SAFE IMAGE HANDLER
# ===========================================

def resize_and_validate(img_data, max_size=800):
    try:
        img = PILImage.open(io.BytesIO(img_data))
        img = img.convert("RGB")
        img.thumbnail((max_size, max_size))

        out = io.BytesIO()
        img.save(out, format='JPEG', quality=90)
        return out.getvalue()

    except UnidentifiedImageError:
        print("\n❌ ERROR: Uploaded file is NOT a valid image.")
        print("✅ Please upload a JPG, JPEG, PNG or WEBP file only.\n")
        return None


# ===========================================
# 📸 STEP 1: Upload default thumbnail
# ===========================================
print("📸 Upload a DEFAULT JPG/PNG thumbnail for ALL MP3 files:")
while True:
    image_upload = files.upload()
    img_name, default_img_data = next(iter(image_upload.items()))

    validated = resize_and_validate(default_img_data)

    if validated:
        default_img_data = validated
        break
    else:
        print("⚠️ Re-upload a valid image file...\n")

display(Image(data=default_img_data, width=150))


# ===========================================
# 🎵 STEP 2: Upload MP3 files
# ===========================================
print("\n⬆️ Upload up to 50 MP3 files:")
upload = files.upload()
mp3_files = [f for f in upload.keys() if f.lower().endswith(".mp3")][:50]
print(f"✅ Uploaded {len(mp3_files)} MP3 files.")


# ===========================================
# 🧠 Helper: Replace cover (erase old)
# ===========================================
def embed_thumbnail_eyed3(mp3_file, image_data):
    audiofile = eyed3.load(mp3_file)
    if audiofile.tag is None:
        audiofile.initTag()

    audiofile.tag.images.remove_all()
    audiofile.tag.images.set(3, image_data, "image/jpeg", u"Cover")
    audiofile.tag.save(version=eyed3.id3.ID3_V2_3)


# Apply default cover
for f in mp3_files:
    embed_thumbnail_eyed3(f, default_img_data)

print("✅ New thumbnail applied to all MP3s (old ones erased).")


# ===========================================
# 💿 BULK CONTROLS
# ===========================================
bulk_album_widget = widgets.Text(value='', description='Album:', layout=widgets.Layout(width='280px'))
bulk_album_btn = widgets.Button(description='Apply Album to All', button_style='warning')
bulk_img_btn  = widgets.Button(description='📸 Change BULK Thumbnail', button_style='danger')

display(widgets.HBox([bulk_album_widget, bulk_album_btn, bulk_img_btn]))


# ===========================================
# 🧠 Parse filename for tags
# ===========================================
def parse_filename(filename):
    name = re.sub(r'\.\w+$', '', filename)
    parts = re.split(r'\s*-\s*', name)
    parts = [p.strip() for p in parts if p.strip()]

    if len(parts) >= 3:
        return parts[1], " - ".join(parts[2:])
    elif len(parts) == 2:
        return parts[0], parts[1]
    else:
        return "", parts[0] if parts else filename


# ===========================================
# 📝 Step 5: Per-file UI
# ===========================================
file_widgets = []

for f in mp3_files:
    parsed_artist, parsed_title = parse_filename(f)

    title_widget  = widgets.Text(value=parsed_title, description='Title:', layout=widgets.Layout(width='220px'))
    artist_widget = widgets.Text(value=parsed_artist, description='Artist:', layout=widgets.Layout(width='220px'))
    album_widget  = widgets.Text(value='', description='Album:', layout=widgets.Layout(width='220px'))

    img_widget = widgets.Image(value=default_img_data, format='jpeg', width=80, height=80)
    new_img_data = {'data': None}

    def make_upload_callback(img_w, store):
        def on_upload(b):
            while True:
                uploaded = files.upload()
                if not uploaded:
                    return

                _, data = next(iter(uploaded.items()))
                validated = resize_and_validate(data)

                if validated:
                    img_w.value = validated
                    store['data'] = validated
                    break
                else:
                    print("⚠️ Re-upload a VALID image only...")

        return on_upload

    upload_btn = widgets.Button(description="📤 Change Thumbnail")
    upload_btn.on_click(make_upload_callback(img_widget, new_img_data))

    box = widgets.HBox([
        img_widget,
        widgets.Label(f, layout=widgets.Layout(width='220px')),
        title_widget,
        artist_widget,
        album_widget,
        upload_btn
    ])

    file_widgets.append((f, title_widget, artist_widget, album_widget, new_img_data, box))


display(widgets.Label("📝 Edit your songs:"))
for item in file_widgets:
    display(item[-1])


# ===========================================
# BULK ALBUM
# ===========================================
def on_bulk_album_clicked(b):
    for _, _, _, album_w, _, _ in file_widgets:
        album_w.value = bulk_album_widget.value.strip()

bulk_album_btn.on_click(on_bulk_album_clicked)


# ===========================================
# BULK IMAGE CHANGE (ERASE OLD)
# ===========================================
def on_bulk_image_click(b):
    global default_img_data

    while True:
        uploaded = files.upload()
        _, data = next(iter(uploaded.items()))
        validated = resize_and_validate(data)

        if validated:
            default_img_data = validated
            break
        else:
            print("⚠️ Upload a valid image...")

    for f, _, _, _, new_img, _ in file_widgets:
        new_img['data'] = default_img_data
        embed_thumbnail_eyed3(f, default_img_data)

    print("✅ Bulk thumbnail replaced & old covers erased")

bulk_img_btn.on_click(on_bulk_image_click)


# ===========================================
# SAVE + ZIP
# ===========================================
save_btn = widgets.Button(description="💾 Save All", button_style='success')
zip_btn  = widgets.Button(description="⬇️ Download ZIP", button_style='info')
out      = widgets.Output()
display(save_btn, zip_btn, out)


def clean_filename(filename):
    return re.sub(r"\s*\(\d+\)(?=\.\w+$)", "", filename)


def on_save_clicked(b):
    for f, title_w, artist_w, album_w, img_data, _ in file_widgets:
        audiofile = eyed3.load(f)

        audiofile.tag.title  = title_w.value.strip()
        audiofile.tag.artist = artist_w.value.strip()
        audiofile.tag.album  = album_w.value.strip()

        cover = img_data['data'] if img_data['data'] else default_img_data

        audiofile.tag.images.remove_all()
        audiofile.tag.images.set(3, cover, "image/jpeg", u"Cover")
        audiofile.tag.save(version=eyed3.id3.ID3_V2_3)

    with out:
        out.clear_output()
        print("✅ All MP3 tags & covers saved successfully")


def on_zip_clicked(b):
    name = f"edited_mp3s_{datetime.now().strftime('%Y%m%d_%H%M')}.zip"

    with zipfile.ZipFile(name, "w") as z:
        for f, *_ in file_widgets:
            z.write(f, clean_filename(f))

    files.download(name)


save_btn.on_click(on_save_clicked)
zip_btn.on_click(on_zip_clicked)

print("\n✅ READY — Safe, Stable, No Image Errors")
